In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from deap import algorithms, base, creator, tools

In [4]:
X=[[0,0], [0,1], [1,0], [1,1]]
Y=[[0],[1],[1],[0]]

In [5]:
model = keras.models.Sequential()
model.add(layers.Dense(5, input_shape=(2,), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.1))

In [7]:
model.predict(X)

array([[0.5       ],
       [0.39044598],
       [0.4938762 ],
       [0.3866255 ]], dtype=float32)

In [8]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(2, 5) dtype=float32, numpy=
 array([[ 0.3413484 ,  0.19459867, -0.3319553 ,  0.02361268, -0.08398724],
        [-0.56094015, -0.9225579 , -0.46661925,  0.65405416, -0.49264166]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(5, 1) dtype=float32, numpy=
 array([[-0.35610175],
        [ 0.58139896],
        [ 0.25370908],
        [-0.6810415 ],
        [ 0.62734294]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [9]:
class WeightsShaper:
    def __init__(self, weights):
        self.shapes=[]
        self.lengths=[]
        for w in weights:
            self.shapes.append(w.numpy().shape)
            self.lengths.append(w.numpy().reshape(-1).shape[0])
    def flat(self, weights):
        flatted=[]
        for w in flatten(weights):
            flatted+=w.numpy().reshape(-1).tolist()
        return array(flatted)    
    
    def unflat(self, flatted, weights):
        pos=0
        for i in range(len(weights)):
            weights[i].assign(flatted[pos:pos+self.lengths[i]].reshape(self.shapes[i]))
            pos+=self.lengths[i]

In [10]:
shaper=WeightsShaper(model.weights)

In [11]:
creator.create("Minimalizator", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Minimalizator)

In [12]:
def eval(individual):
    shaper.unflat(array(individual), model.weights)
    loss=model.evaluate(X,Y, verbose=0)
#     loss=sum(abs(model.predict(X)-array(Y)))
    return loss, 

In [13]:
popSize=100
indSize=sum(shaper.lengths)

In [14]:
toolbox = base.Toolbox()

toolbox.register("gene", random.uniform, -1, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gene, indSize)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [15]:
pop = toolbox.population(n=popSize)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [26]:
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=30, halloffame=hof, verbose=True, stats=stats)

gen	nevals	avg       	std       	min       	max      
0  	0     	0.00671727	0.00183207	0.00468457	0.0201298
1  	68    	0.0197403 	0.0888556 	0.00407536	0.866825 
2  	53    	0.0426622 	0.306721  	0.00301104	3.06836  
3  	73    	0.0134813 	0.0832359 	0.00282493	0.841523 
4  	67    	0.0189227 	0.120321  	0.00205311	1.20756  
5  	57    	0.0591692 	0.493836  	0.00205311	4.95418  
6  	63    	0.00509718	0.0102686 	0.00155345	0.100057 
7  	63    	0.00853996	0.0522222 	0.00155345	0.527828 
8  	54    	0.00372448	0.00810559	0.00133145	0.0706859
9  	63    	0.0380947 	0.250853  	0.00110027	1.90261  
10 	67    	0.0132208 	0.102556  	0.000954717	1.02735  
11 	55    	0.00424054	0.0262838 	0.000883394	0.265698 
12 	66    	0.0448853 	0.271924  	0.000723048	2.15378  
13 	60    	0.00838429	0.0536651 	0.000565183	0.523342 
14 	68    	0.00355068	0.0214442 	0.000565183	0.213789 
15 	56    	0.0177965 	0.132141  	0.000532506	1.31398  
16 	63    	0.00819214	0.074561  	0.000532506	0.750064 
17 	68    	0.0180184 

In [27]:
hof[0]

[-3.126956884093955,
 3.198300371436284,
 2.0261400401917777,
 -2.449050204972247,
 -2.0646531887947974,
 1.6886925338332837,
 2.011451159506794,
 -2.585183229577143,
 -1.9334729300539837,
 -0.5286607989488941,
 0.08778797427963797,
 -0.9738217740667816,
 -0.028323279442915544,
 1.9816437576210988,
 0.41779596629085575,
 7.6236316644311835,
 -2.23233032347526,
 8.486284380316922,
 -6.178250651277026,
 -0.5151129011545427,
 -1.7786981303617817]

In [28]:
shaper.unflat(array(hof[0]), model.weights)
model.predict(X)

array([[1.2811666e-06],
       [9.9989378e-01],
       [9.9996316e-01],
       [1.3207938e-05]], dtype=float32)

In [29]:
X_test=[[0,1], [0,0], [1,1], [1,0]]
model.predict(X_test)

array([[9.9989378e-01],
       [1.2811666e-06],
       [1.3207938e-05],
       [9.9996316e-01]], dtype=float32)